In [ ]:
%%html
<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [17]'.</span>

An Exception was encountered at 'In [17]'.

In [1]:
import tensorflow as tf
import glob
import nibabel as nib
import numpy as np

import pandas as pd

tf.__version__

'2.0.0'

In [2]:
train_df = pd.read_csv('/data2/csv/intell_residual_train.csv')
val_df = pd.read_csv('/data2/csv/intell_residual_valid.csv')
test_df = pd.read_csv('/data2/csv/intell_residual_test.csv')

In [3]:
#train_df = train_df[train_df['abcd_site']==16]
#val_df = val_df[val_df['abcd_site']==16]
#test_df = test_df[test_df['abcd_site']==16]

In [4]:
patht1 = './data_T1_lowerres/'
patht2 = './data_T2_lowerres/'
pathdef = './data_defusion_lowerres/'

In [5]:
patht1 = '/data/home/benedikt_d_schifferer/data_T1_T2_201909/'
patht2 = '/data/home/benedikt_d_schifferer/data_T1_T2_201909/'
#pathdef = './data_def_lowerres_cropped/'

In [6]:
all_files = glob.glob(patht1 + '*_T1.nii.gz') + glob.glob(patht2 + '*_T2.nii.gz')

In [7]:
def extract_subject_id(text):
    text_split = text.split('sub-')[1]
    text_split = text_split.split('_', 1)
    return(text_split)

In [8]:
all_subjects = [extract_subject_id(x)[0] for x in all_files]

In [9]:
all_subjects = list(dict.fromkeys(all_subjects))

In [10]:
len(all_subjects)

11373

In [11]:
def get_filenames(sub, all_files):
    return([x for x in all_files if sub in x])

In [12]:
get_filenames(all_subjects[0], all_files)

['/data/home/benedikt_d_schifferer/data_T1_T2_201909/sub-NDARINVU8D0FE3D_T1.nii.gz']

In [13]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def parse_subject(subject, files):
    for file in files:
        img = nib.load(file)
        data = np.array(img.dataobj)
        if '_T1.' in file:
            t1 = data.copy()
            if t1.shape != (256,256,256):
                print('error')
        if '_T2.' in file:
            t2 = data.copy()
            if t2.shape != (256,256,256):
                print('error')
        if '_AD.' in file:
            ad = data.copy()
            if ad.shape != (64,64,64):
                print('error')
        if '_FA.' in file:
            fa = data.copy()
            if fa.shape != (64,64,64):
                print('error')
        if '_RD.' in file:
            rd = data.copy()
            if rd.shape != (64,64,64):
                print('error')
        if '_MD.' in file:
            md = data.copy()
            if md.shape != (64,64,64):
                print('error')
    example = tf.train.Example(features = tf.train.Features(
        feature = {
            't1':_bytes_feature(t1.tostring()),
            't2':_bytes_feature(t2.tostring()),
            'subjectid':_bytes_feature(subject.encode('utf-8'))
        }))
    return(example)

def convert_to_records(all_subjects, all_files, sample=100, path = 'test4.tfrecords'):
    print('writing to {}'.format(path))
    counter = 0
    with tf.io.TFRecordWriter(path) as writer:
        for i in range(min(len(all_subjects), sample)):
            subjectid = all_subjects[i]
            files = get_filenames(subjectid, all_files)
            if len(files)==2:
                example = parse_subject(subjectid, files)
                writer.write(example.SerializeToString())
                if i%100==0:
                    print('writing {}th image'.format(i))
            else:
                print(subjectid)
                print(files)
                print('missing images')
                counter += 1
    print(counter)

In [14]:
train_df['subjectkey'] = train_df['subjectkey'].str.replace('_', '')
val_df['subjectkey'] = val_df['subjectkey'].str.replace('_', '')
test_df['subjectkey'] = test_df['subjectkey'].str.replace('_', '')

In [15]:
import random

random.shuffle(all_subjects)

In [16]:
#convert_to_records([x for x in all_subjects if x in list(train_df['subjectkey'])], all_files, sample=100000, path = '/data2/t1t2_train_allimages_256_v4.tfrecords')
#convert_to_records([x for x in all_subjects if x in list(val_df['subjectkey'])], all_files, sample=100000, path = '/data2/t1t2_val_allimages_256_v4.tfrecords')
convert_to_records([x for x in all_subjects if x in list(test_df['subjectkey'])], all_files, sample=100000, path = '/data2/t1t2_test_allimages_256_v4_new.tfrecords')

writing to /data2/t1t2_test_allimages_256_v4_new.tfrecords


writing 0th image


writing 100th image


writing 200th image


writing 300th image


...

writing 900th image


writing 1000th image


writing 1100th image


writing 1200th image


0


In [17]:
error

NameError: name 'error' is not defined

In [ ]:
train_files = [x for x in all_subjects if x in list(train_df['subjectkey'])]

In [ ]:
for subjectid in train_files:
    files = get_filenames(subjectid, all_files)
    for file in files:
        img = nib.load(file)
        data = np.array(img.dataobj)
        if '_T1.' in file:
            t1 = data.copy()
            if t1.shape != (64,64,64):
                print('t1 shape')
                print('error')
                print(file)
            if t1.dtype != 'uint8':
                print('t1 type')
                print('error')
                print(file)
        if '_T2.' in file:
            t2 = data.copy()
            t2 = t2.astype(np.float32)
            if t2.shape != (64,64,64):
                print('t2 shape')
                print('error')
                print(file)
            if t2.dtype != 'float32':
                print('t2 type')
                print('error')
                print(file)

In [ ]:
t2.astype(np.float32).dtype == 'float32'

In [ ]:
!du -sh t1t2_train_sample100.tfrecords

In [ ]:
tfDataSet = tf.data.TFRecordDataset('t1t2_val_site16_allimages_v4.tfrecords')

In [ ]:
read_features = {
    't1': tf.io.FixedLenFeature([], dtype=tf.string),
    't2': tf.io.FixedLenFeature([], dtype=tf.string),
    'ad': tf.io.FixedLenFeature([], dtype=tf.string),
    'fa': tf.io.FixedLenFeature([], dtype=tf.string),
    'md': tf.io.FixedLenFeature([], dtype=tf.string),
    'rd': tf.io.FixedLenFeature([], dtype=tf.string),
    'subjectid': tf.io.FixedLenFeature([], dtype=tf.string)
}


def _parse_(serialized_example, decoder = np.vectorize(lambda x: x.decode('UTF-8'))):
    example = tf.io.parse_single_example(serialized_example, read_features)
    t1 = tf.reshape(tf.io.decode_raw(example['t1'], tf.int8), (64,64,64))
    t2 = tf.reshape(tf.io.decode_raw(example['t2'], tf.float32), (64,64,64))
    ad = tf.reshape(tf.io.decode_raw(example['ad'], tf.float32), (64,64,64))
    fa = tf.reshape(tf.io.decode_raw(example['fa'], tf.float32), (64,64,64))
    md = tf.reshape(tf.io.decode_raw(example['md'], tf.float32), (64,64,64))
    rd = tf.reshape(tf.io.decode_raw(example['rd'], tf.float32), (64,64,64))
    subjectid = example['subjectid']
    return ({'t1': t1, 't2': t2, 'ad': ad, 'fa':fa, 'md': md, 'rd': rd,'subjectid': subjectid})

tfrecord_dataset = tfDataSet.map(lambda x:_parse_(x)).shuffle(True).batch(32)

In [ ]:
a = iter(tfrecord_dataset)

In [ ]:
for b in tfrecord_dataset:
    pass

In [ ]:
a = next(iter(tfrecord_dataset))

In [ ]:
a['fa'].shape

In [ ]:
decoder = np.vectorize(lambda x: x.decode('UTF-8'))

decoder(a['subjectid'].numpy())

In [ ]:
tf.strings.unicode_decode(a['subjectid'], input_encoding='UTF-8')